In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2
import asyncio

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.core.series import Quote
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.core.utils import recognize_time

from qubx.impl.ccxt_connector import CCXTDataConnector
from qubx.impl.ccxt_trading import CCXTSyncTradingConnector
from qubx.utils.runner import get_account_config

def Q(time: str, bid: float, ask: float) -> Quote:
    return Quote(recognize_time(time), bid, ask, 0, 0)

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        
2024-07-11 22:56:21.815 [ ℹ️ ] NTP offset controller thread is started


In [2]:
class TestStrategy(IStrategy):
    capital_invested: int = 1000

    def on_start(self, ctx: StrategyContext):
        logger.info(f"Strategy started | {ctx.time()}")

    def on_fit(self, ctx: 'StrategyContext', fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f"> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        logger.info(f"{event.time} -> {event}")
        return None

    def ohlcs(self, timeframe:str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def on_stop(self, ctx: StrategyContext):
        logger.info(f"> test is stopped")

    def tracker(self, ctx: StrategyContext) -> PortfolioRebalancerTracker:
        return PortfolioRebalancerTracker(ctx, 1000, 0)

# Simulated broker and paper trading

In [3]:
from qubx.backtester.simulator import SimulatedExchangeService

In [4]:
auth = get_account_config('binance-qbx', './configs/.env')

sbroker = SimulatedExchangeService('binance', 10000, 'vip0_usdt', 'USDT')
data = CCXTDataConnector('binance', sbroker, **auth)

instruments = [lookup.find_symbol('BINANCE', 'BTCUSDT')]
ctx = StrategyContext(
    TestStrategy(), dict(
        capital_invested=100,
        trading_allowed=False
    ), 
    data, sbroker, instruments=instruments,  # type: ignore
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=0),
    trigger_spec='1Min -5Sec',
)
S = ctx.strategy

2024-07-11 12:05:09.655 [ ℹ️ ] binance initialized - current time 1970-01-01T00:00:00.000000000
2024-07-11 12:05:09.656 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-11 12:05:09.657 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-11 12:05:09.658 [ 🐞 ] Triggering strategy on every 1Min bar after 55S
2024-07-11 12:05:09.656 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-11 12:05:09.657 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-11 12:05:09.658 [ 🐞 ] Triggering strategy on every 1Min bar after 55S


In [5]:
ctx.start()

2024-07-11 12:05:10.906 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 0} for 
	['BTCUSDT'] 
2024-07-11 12:05:10.906 [ ℹ️ ] Subscribed on ohlc updates for 1 symbols: 
		['BTCUSDT']
2024-07-11 12:05:10.916 [ ℹ️ ] Strategy started | 1970-01-01T00:00:00.000000000
2024-07-11 12:05:10.917 [ ℹ️ ] (StrategyContext) Start processing market data
2024-07-11 12:05:10.918 [ ℹ️ ] (StrategyContext) strategy is started


2024-07-11 12:05:21.011 [ 🐞 ] [2024-07-11T09:05:21.233605376]: Invoking TestStrategy on_fit('1970-01-01 00:00:00', 'None')
2024-07-11 12:05:21.013 [ ℹ️ ] > Fit is called | fit_time: 1970-01-01 00:00:00 / prev: None
2024-07-11 12:05:21.013 [ 🐞 ] [2024-07-11T09:05:21.233605376]: TestStrategy is fitted
2024-07-11 12:05:55.945 [ ℹ️ ] 2024-07-11T09:05:56.272944896 -> TriggerEvent(time=numpy.datetime64('2024-07-11T09:05:56.272944896'), type='bar', instrument=Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE', base='BTC', quote='USDT', margin_symbol=None, min_tick=0.01, min_size_step=1e-05, min_size=1e-05, futures_info=None, _aux_instrument=None), data={o:58238.010000 | h:58272.000000 | l:58214.010000 | c:58270.000000 | v:338529.375459})
2024-07-11 12:06:55.254 [ ℹ️ ] 2024-07-11T09:06:55.503642880 -> TriggerEvent(time=numpy.datetime64('2024-07-11T09:06:55.503642880'), type='bar', instrument=Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE', base='BTC', q

In [12]:
data.get_historical_ohlcs('BTCUSDT', '5Min', 200)

[{o:57959.030000 | h:57965.690000 | l:57868.460000 | c:57868.470000 | v:2697017.716737},
 {o:57868.470000 | h:57885.120000 | l:57815.710000 | c:57881.660000 | v:4414635.491012},
 {o:57881.660000 | h:57885.120000 | l:57645.000000 | c:57645.000000 | v:7407404.272832},
 {o:57645.000000 | h:57707.140000 | l:57584.370000 | c:57685.850000 | v:6804606.883147},
 {o:57685.860000 | h:57713.270000 | l:57662.250000 | c:57696.730000 | v:2274047.146135},
 {o:57696.730000 | h:57710.000000 | l:57594.890000 | c:57659.330000 | v:3732914.305474},
 {o:57659.340000 | h:57659.340000 | l:57574.100000 | c:57578.610000 | v:2860673.431409},
 {o:57578.610000 | h:57713.260000 | l:57578.600000 | c:57713.250000 | v:2973817.488873},
 {o:57713.260000 | h:57755.740000 | l:57590.000000 | c:57621.050000 | v:2904756.301209},
 {o:57621.060000 | h:57668.680000 | l:57534.030000 | c:57540.410000 | v:3435904.738911},
 {o:57539.520000 | h:57632.420000 | l:57439.670000 | c:57623.070000 | v:9585137.148261},
 {o:57623.060000 | h:

In [80]:
print(sbroker.get_position(instruments[0]))
print(sbroker.get_orders())

2024-07-11 09:12:04 [BINANCE:BTCUSDT]       0.00000      0.00    +9.6170 | 58227.99        0.00
[]


In [53]:
ctx.trade('BTCUSDT', 0.1)

2024-07-11 12:11:35.820 [ ℹ️ ] (StrategyContext) sending market buy for 0.1 of BTCUSDT ...
2024-07-11 12:11:35.820 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100006 MARKET BUY 0.1 executed at 58214.005
2024-07-11 12:11:35.820 [ ℹ️ ]   ::  traded 0.1 for BTCUSDT @ 58214.005 -> 0.00
2024-07-11 12:11:35.820 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100006 MARKET BUY 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100006', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-11T09:11:34.097511168'), quantity=0.1, price=0, side='BUY', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_6', cost=0.0)

In [79]:
ctx.trade('BTCUSDT', -0.1)

2024-07-11 12:12:04.642 [ ℹ️ ] (StrategyContext) sending market sell for 0.1 of BTCUSDT ...
2024-07-11 12:12:04.642 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100012 MARKET SELL 0.1 executed at 58227.985
2024-07-11 12:12:04.644 [ ℹ️ ]   ::  traded -0.1 for BTCUSDT @ 58227.985 -> 2.10
2024-07-11 12:12:04.644 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100012 MARKET SELL 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100012', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-11T09:12:04.778781440'), quantity=0.1, price=0, side='SELL', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_12', cost=0.0)

In [124]:
ctx.trade('BTCUSDT', 0.1, 45000)

2024-07-11 08:55:43.969 [ ℹ️ ] (StrategyContext) sending limit buy for 0.1 of BTCUSDT ...
2024-07-11 08:55:43.969 [ 🐞 ] [OMS] BTCUSDT - registered SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 45000


Order(id='SIM-ORDER-BTCUSDT-100005', type='LIMIT', symbol='BTCUSDT', time=numpy.datetime64('2024-07-11T05:55:41.389935104'), quantity=0.1, price=45000, side='BUY', status='OPEN', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_5', cost=0.0)

In [136]:
ctx.cancel('BTCUSDT')

2024-07-11 08:56:31.011 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 canceled
2024-07-11 08:56:31.011 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 of BTCUSDT -> CANCELED
2024-07-11 08:56:31.011 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 of BTCUSDT -> CANCELED


In [81]:
ctx.stop()
data.close()

2024-07-11 12:12:14.148 [ ℹ️ ] > test is stopped
2024-07-11 12:12:14.149 [ ℹ️ ] 	CachedMarketDataHolder.update_by_bar took 0.0000254 secs
2024-07-11 12:12:14.149 [ ℹ️ ] 	StrategyContext._processing_bar took 0.0000359 secs
2024-07-11 12:12:14.150 [ ℹ️ ] 	StrategyContext.handler took 0.0000034 secs
2024-07-11 12:12:14.150 [ ℹ️ ] 	StrategyContext._process_incoming_data took 1.0039607 secs
2024-07-11 12:12:14.150 [ ℹ️ ] 	loggers.notify took 0.0000024 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	strategy.on_fit took 0.0014524 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	strategy.on_event took 0.0012006 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	StrategyContext.trade took 0.0033409 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	StrategyContext._processing_order took 0.0006063 secs
2024-07-11 12:12:14.151 [ ℹ️ ] 	StrategyContext._processing_deals took 0.0011736 secs
2024-07-11 12:12:14.151 [ ❌ ] This event loop is already running


c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\impl\ccxt_connector.py:288: RuntimeWarning: coroutine 'Exchange.close' was never awaited
  def get_quote(self, symbol: str) -> Optional[Quote]:


# Simulated data provider

In [144]:
r = CsvStorageDataReader("../tests/data/csv")
# r = MultiQdbConnector('xlydian-data')

In [145]:
r.get_names()

['AAPL',
 'BTCUSDT_ohlcv_M1',
 'BTCUSDT_ohlcv_M1_sec',
 'ETHUSDT',
 'FDI_test',
 'quotes',
 'SOLUSDT',
 'SPY',
 'SPY1',
 'BINANCE.UM:BTCUSDT',
 'BINANCE.UM:SOLUSDT']

In [ ]:
# r.get_names('candles')

In [146]:
instr = lookup.find_symbol("BINANCE.UM", "SOLUSDT")
# stream = r.read('SOLUSDT', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick, timestamp_units='ms'))
stream = r.read('BINANCE.UM:SOLUSDT', '2024-04-01', '2024-04-10', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick, timestamp_units='ms'))
# stream = r.read('BINANCE.UM:BTCUSDT', '2024-01-01', '2024-02-10', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick, timestamp_units='ms'))

In [147]:
stream[0], stream[-1]

([2024-04-01T00:00:00.000006000]	202.68450 (1000000000.0) | 202.68550 (1000000000.0),
 [2024-04-02T09:18:00.000054000]	182.66950 (1000000000.0) | 182.67050 (1000000000.0))

In [150]:
from qubx.backtester.simulator import SimulatedExchangeService, SimulatedDataProvider, DataLoader

In [151]:
dp = DataLoader(
    RestoreTicksFromOHLC(trades=False, spread=instr.min_tick, timestamp_units='ms'), 
    r, instr, '1Min', 10
)

In [152]:
sbroker = SimulatedExchangeService('binance', 10000, 'vip0_usdt', 'USDT')
sdp = SimulatedDataProvider('binance', sbroker, r)

2024-07-12 00:32:25.206 [ ℹ️ ] SimulatedData.binance initialized


In [153]:
instruments = [lookup.find_symbol('BINANCE.UM', 'BTCUSDT')]
sdp.subscribe('ohlc', instruments, '15Min')

True

In [154]:
sdp._pumps['BTCUSDT'].load('2024-01-01', '2024-02-01')

[[2024-01-01T00:00:06.000000000]	42313.95000 (1000000000.0) | 42314.05000 (1000000000.0),
 [2024-01-01T00:00:24.000000000]	42289.55000 (1000000000.0) | 42289.65000 (1000000000.0),
 [2024-01-01T00:00:36.000000000]	42335.75000 (1000000000.0) | 42335.85000 (1000000000.0),
 [2024-01-01T00:00:54.000000000]	42331.85000 (1000000000.0) | 42331.95000 (1000000000.0),
 [2024-01-01T00:01:06.000000000]	42331.85000 (1000000000.0) | 42331.95000 (1000000000.0),
 [2024-01-01T00:01:24.000000000]	42331.75000 (1000000000.0) | 42331.85000 (1000000000.0),
 [2024-01-01T00:01:36.000000000]	42353.05000 (1000000000.0) | 42353.15000 (1000000000.0),
 [2024-01-01T00:01:54.000000000]	42350.35000 (1000000000.0) | 42350.45000 (1000000000.0),
 [2024-01-01T00:02:06.000000000]	42350.35000 (1000000000.0) | 42350.45000 (1000000000.0),
 [2024-01-01T00:02:24.000000000]	42349.55000 (1000000000.0) | 42349.65000 (1000000000.0),
 [2024-01-01T00:02:36.000000000]	42370.75000 (1000000000.0) | 42370.85000 (1000000000.0),
 [2024-01-